In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# 허태명님과 김태진님의 커널을 참고했습니다
https://www.kaggle.com/tmheo74/3rd-ml-month-car-image-cropping

https://www.kaggle.com/fulrose/3rd-ml-month-car-model-classification-baseline/data#3rd-ML-Month---Car-Model-Classification-

In [ ]:
print(os.listdir("../input/cropped-car-image"))

In [ ]:
COMPETITION_DATA_PATH = '../input/2019-3rd-ml-month-with-kakr'
IMAGE_DATA_PATH = '../input/cropped-car-image'

# 이미지 폴더 경로
TRAIN_IMG_PATH = os.path.join(IMAGE_DATA_PATH, 'train_crop')
TEST_IMG_PATH = os.path.join(IMAGE_DATA_PATH, 'test_crop')

# CSV 파일 경로
df_train = pd.read_csv(os.path.join(COMPETITION_DATA_PATH, 'train.csv'))
df_test = pd.read_csv(os.path.join(COMPETITION_DATA_PATH, 'test.csv'))
df_class = pd.read_csv(os.path.join(COMPETITION_DATA_PATH, 'class.csv'))

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_class.head()

In [ ]:
# 트레이닝, 테스팅 이미지갯수를 확인하겠습니다
print(len(os.listdir(TRAIN_IMG_PATH)))
print(len(os.listdir(TEST_IMG_PATH)))

In [ ]:
# 클래스 갯수를 확인합니다
print(df_class.shape[0])

In [ ]:
# 이미지들을 구경해 보겠습니다
from PIL import Image, ImageDraw

filenames = df_train.img_file[16:20]
filenames.head()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))

for index, filename in enumerate(filenames):
    img = Image.open(TRAIN_IMG_PATH + '/' + filename)
    plt.subplot(2,2,index+1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(filename)

In [ ]:
# 사진이 찍힌 각도도 많이 다르고, 조명, 기울기, 등등 다른요소들이 많네요

In [ ]:
# 모델링
from sklearn.model_selection import train_test_split

df_train['class'] = df_train['class'].astype('str')

# bounding box 컬럼들을 제거해줍니다
df_train = df_train[['img_file', 'class']]
df_test = df_test[['img_file']]

df_test.head(1)

In [ ]:
df_train.head(1)

In [ ]:
index_list = np.arange(df_train.shape[0])
print(index_list)

In [ ]:
# 이번 대회는 micro f1 score 를 기준으로 평가되기때문에 몇몇 클래스에서만 좋은 정확도를 내는것보다 모든 클래스에서 골고루 높은 정확도를 내는게
# 점수가 더 높을거 같네요.
# 트레이닝 데이터셋에 클래스분포를보고 그에맞춰서 training/validation 데이타셋으로 나눠주는게 좋을거같아요.

df_train['class'].value_counts().head()

In [ ]:
class_labels = list(df_train['class'])

In [ ]:
# applying stratification
train_idx, val_idx = train_test_split(index_list, stratify=class_labels, train_size=0.8, random_state=42)

X_train = df_train.iloc[train_idx, :]
X_val = df_train.iloc[val_idx, :]

print(X_train.shape)
print(X_val.shape)
print(df_test.shape)

In [ ]:
X_train.head()

In [ ]:
X_train['class'].value_counts().head()

In [ ]:
X_val['class'].value_counts().head()

In [ ]:
# Image Generator
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Parameter
img_size = (224, 224)
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
nb_test_samples = len(df_test)
epochs = 20
batch_size = 32

# Define Generator config
train_datagen = ImageDataGenerator(
    horizontal_flip = True,
    vertical_flip = False,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest',
    zoom_range = 0.10,
    rotation_range = 20,
    preprocessing_function=preprocess_input)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Make Generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe = X_train, 
    directory = TRAIN_IMG_PATH,
    x_col = 'img_file',
    y_col = 'class',
    target_size = img_size,
    color_mode ='rgb',
    class_mode ='categorical',
    batch_size =batch_size,
    seed = 42
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=X_val, 
    directory= TRAIN_IMG_PATH,
    x_col = 'img_file',
    y_col = 'class',
    target_size = img_size,
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe = df_test,
    directory = TEST_IMG_PATH,
    x_col = 'img_file',
    y_col = None,
    target_size = img_size,
    color_mode = 'rgb',
    class_mode = None,
    batch_size = batch_size,
    shuffle = False
)

In [ ]:
# loading pretrained model
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout

model = Sequential()
model.add(resnet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(196, activation='softmax'))
model.summary()

In [ ]:
from keras import optimizers

optimizer = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0 :
        return (num_samples // batch_size) + 1
    else:
        return num_samples // batch_size
    
from keras.callbacks import ModelCheckpoint, EarlyStopping

filepath = "resnet_model_{val_acc:.2f}_{val_loss:.4f}.h5"

ckpt = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True)
es = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

callbackList = [ckpt, es]

history = model.fit_generator(
    train_generator,
    steps_per_epoch = get_steps(nb_train_samples, batch_size),
    epochs=epochs,
    validation_data = validation_generator,
    validation_steps = get_steps(nb_validation_samples, batch_size),
    callbacks = callbackList
)
gc.collect()